In [1]:
#GPU count and name
!nvidia-smi -L

#use this command to see GPU activity while doing Deep Learning tasks, for this command 'nvidia-smi' and for above one to work, go to 'Runtime > change runtime type > Hardware Accelerator > GPU'
!nvidia-smi

!lscpu |grep 'Model name'

#no.of sockets i.e available slots for physical processors
!lscpu | grep 'Socket(s):'

#no.of cores each processor is having 
!lscpu | grep 'Core(s) per socket:'

#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'

!lscpu | grep "L3 cache"

#if it had turbo boost it would've shown Min and Max MHz also but it is only showing current frequency this means it always operates at 2.3GHz
!lscpu | grep "MHz"

#memory that we can use
!free -h --si | awk  '/Mem:/{print $2}'

#hard disk space that we can use
!df -h / | awk '{print $4}'

GPU 0: Tesla T4 (UUID: GPU-7c31027e-37f9-5ef6-dc62-bbb892196714)
Wed Jan 20 05:32:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                       

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
import time

In [5]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
import datetime
import pandas as pd
import numpy as np

In [6]:
downloaded = drive.CreateFile({'id':"1kvpS14EKmAOviVoqvRSZ3yYpNs-eGSp-"})   # replace the id with id of file you want to access
downloaded.GetContentFile('Modified_data.csv')

In [6]:
dataset = pd.read_csv('Modified_data.csv')

In [7]:
dataset

,Local time,Open,High,Low,Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,...,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,Log_Diff
0,08.05.2017 03:59:00.000 GMT+0400,1555,1590,1554,1590,0.1280,0.128000,0.1280,1.000000,0.000000,50.000000,0.000000e+00,0.000000e+00,-0.104849,1.000000e+03,1578.000000,0.000000,1590.00,1590.000000,1590.000000,0.000000,0.000000,0.0,0.0,1578.000000,1614.000000,1542.000000,4.562738,0.666667,0.0,0.0,1554.0,1590.0,1572.0,2.264151,1.000000,0.000000,0.000000,0.000000,0.000000,...,-81.943519,1.000000,0.000000,7216.223846,-819.445880,-819.445880,0.000000,1572.0,1572.0,1572.00,1572.0,8805.37337,8803.83053,4.0,4.0,0.0,-1.000000,-1.000000,0.0,0.0,0.00000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,100.000000,-0.000000,0.000000,1590.000000,0.000000,0.000000,0.000000,0.000000,-81.944588,0.000000,0.000000,0.022258
1,08.05.2017 04:00:00.000 GMT+0400,1555,1601,1554,1592,2.1672,1.465209,2.2952,0.638379,4.334400,100.000000,1.192783e+10,1.192783e+10,-0.102163,1.000000e+03,1582.091670,0.000000,1591.00,1593.000000,1589.000000,0.251414,0.750000,0.0,0.0,1580.166667,1621.666667,1538.666667,5.252610,0.642570,0.0,0.0,1554.0,1601.0,1577.5,2.954117,0.808511,0.000000,0.159544,0.031909,0.127635,...,0.000246,2.048299,66.666667,7215.223846,-819.332324,-819.389102,0.056778,1577.5,1577.5,1577.50,1577.5,8805.37337,8803.83053,8.0,4.0,4.0,-1.000000,-1.000000,0.0,0.0,0.00000,100.000000,0.000000,0.000000,0.000000,100.000000,45.783133,80.851064,90.425532,-19.148936,0.000000,1590.890367,0.000000,58.294265,11.658853,46.635412,0.125786,0.125707,0.125786,0.023516
2,08.05.2017 04:01:00.000 GMT+0400,1592,1592,1591,1591,1.7247,-0.259491,0.5705,-0.064552,3.468814,100.000000,8.117354e+08,6.369784e+09,0.001643,9.993719e+02,1586.056718,0.000000,1591.00,1592.632993,1589.367007,0.205279,0.500000,0.0,0.0,1583.888889,1611.888889,1555.888889,3.535602,0.626984,0.0,0.0,1554.0,1601.0,1577.5,2.954117,0.787234,0.000000,0.202953,0.066118,0.136835,...,0.000522,2.919588,100.000000,7215.223846,-819.332324,-819.370176,0.037852,1577.5,1577.5,1577.50,1577.5,8805.37337,8803.83053,8.0,4.0,4.0,-1.000000,1590.000000,0.0,1.0,0.00000,65.000000,0.000000,0.000000,0.000000,98.895028,45.238095,78.723404,86.524823,-21.276596,0.000000,1590.939209,0.000000,65.512320,22.429546,43.082774,-0.062814,-0.062834,0.062893,-0.000628
3,08.05.2017 04:02:00.000 GMT+0400,1591,1592,1589,1591,3.3491,0.856875,3.9196,0.116281,2.973269,53.680096,-8.957630e+07,4.216664e+09,-0.001083,9.993719e+02,1588.151870,0.000000,1591.00,1592.414214,1589.585786,0.177777,0.500000,0.0,0.0,1585.583333,1607.333333,1563.833333,2.743470,0.624521,0.0,0.0,1554.0,1601.0,1577.5,2.954117,0.787234,0.000000,0.234650,0.099824,0.134826,...,0.000806,3.676931,62.564103,7215.223846,-819.332324,-819.360713,0.028389,1577.5,1577.5,1577.50,1577.5,8805.37337,8803.83053,8.0,4.0,4.0,1591.000000,1590.000000,1.0,0.0,0.00000,65.000000,0.000000,0.000000,0.000000,98.002220,45.977011,78.723404,79.432624,-21.276596,0.000000,1590

In [8]:
dataset = dataset.drop(['Local time'], 1)

In [9]:
dataset

,Open,High,Low,Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,...,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,Log_Diff
0,1555,1590,1554,1590,0.1280,0.128000,0.1280,1.000000,0.000000,50.000000,0.000000e+00,0.000000e+00,-0.104849,1.000000e+03,1578.000000,0.000000,1590.00,1590.000000,1590.000000,0.000000,0.000000,0.0,0.0,1578.000000,1614.000000,1542.000000,4.562738,0.666667,0.0,0.0,1554.0,1590.0,1572.0,2.264151,1.000000,0.000000,0.000000,0.000000,0.000000,1590.000000,...,-81.943519,1.000000,0.000000,7216.223846,-819.445880,-819.445880,0.000000,1572.0,1572.0,1572.00,1572.0,8805.37337,8803.83053,4.0,4.0,0.0,-1.000000,-1.000000,0.0,0.0,0.00000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,100.000000,-0.000000,0.000000,1590.000000,0.000000,0.000000,0.000000,0.000000,-81.944588,0.000000,0.000000,0.022258
1,1555,1601,1554,1592,2.1672,1.465209,2.2952,0.638379,4.334400,100.000000,1.192783e+10,1.192783e+10,-0.102163,1.000000e+03,1582.091670,0.000000,1591.00,1593.000000,1589.000000,0.251414,0.750000,0.0,0.0,1580.166667,1621.666667,1538.666667,5.252610,0.642570,0.0,0.0,1554.0,1601.0,1577.5,2.954117,0.808511,0.000000,0.159544,0.031909,0.127635,1591.000000,...,0.000246,2.048299,66.666667,7215.223846,-819.332324,-819.389102,0.056778,1577.5,1577.5,1577.50,1577.5,8805.37337,8803.83053,8.0,4.0,4.0,-1.000000,-1.000000,0.0,0.0,0.00000,100.000000,0.000000,0.000000,0.000000,100.000000,45.783133,80.851064,90.425532,-19.148936,0.000000,1590.890367,0.000000,58.294265,11.658853,46.635412,0.125786,0.125707,0.125786,0.023516
2,1592,1592,1591,1591,1.7247,-0.259491,0.5705,-0.064552,3.468814,100.000000,8.117354e+08,6.369784e+09,0.001643,9.993719e+02,1586.056718,0.000000,1591.00,1592.632993,1589.367007,0.205279,0.500000,0.0,0.0,1583.888889,1611.888889,1555.888889,3.535602,0.626984,0.0,0.0,1554.0,1601.0,1577.5,2.954117,0.787234,0.000000,0.202953,0.066118,0.136835,1591.000000,...,0.000522,2.919588,100.000000,7215.223846,-819.332324,-819.370176,0.037852,1577.5,1577.5,1577.50,1577.5,8805.37337,8803.83053,8.0,4.0,4.0,-1.000000,1590.000000,0.0,1.0,0.00000,65.000000,0.000000,0.000000,0.000000,98.895028,45.238095,78.723404,86.524823,-21.276596,0.000000,1590.939209,0.000000,65.512320,22.429546,43.082774,-0.062814,-0.062834,0.062893,-0.000628
3,1591,1592,1589,1591,3.3491,0.856875,3.9196,0.116281,2.973269,53.680096,-8.957630e+07,4.216664e+09,-0.001083,9.993719e+02,1588.151870,0.000000,1591.00,1592.414214,1589.585786,0.177777,0.500000,0.0,0.0,1585.583333,1607.333333,1563.833333,2.743470,0.624521,0.0,0.0,1554.0,1601.0,1577.5,2.954117,0.787234,0.000000,0.234650,0.099824,0.134826,1591.000000,...,0.000806,3.676931,62.564103,7215.223846,-819.332324,-819.360713,0.028389,1577.5,1577.5,1577.50,1577.5,8805.37337,8803.83053,8.0,4.0,4.0,1591.000000,1590.000000,1.0,0.0,0.00000,65.000000,0.000000,0.000000,0.000000,98.002220,45.977011,78.723404,79.432624,-21.276596,0.000000,1590.966189,0.000000,74.372407,32.818118,41.554288,0.000000,0.000000,0.062893,0.0000

In [10]:
training_set = dataset.iloc[:len(dataset)-2880, :].values # to get all the features then extract the openinig and closing price
validation_set = dataset.iloc[len(training_set)+1:len(dataset)-1440, :].values
testing_set = dataset.iloc[len(dataset)-1440:, :].values

In [11]:
training_set

array([[ 1.55500000e+03,  1.59000000e+03,  1.55400000e+03, ...,
         0.00000000e+00,  0.00000000e+00,  2.22584706e-02],
       [ 1.55500000e+03,  1.60100000e+03,  1.55400000e+03, ...,
         1.25707119e-01,  1.25786164e-01,  2.35155418e-02],
       [ 1.59200000e+03,  1.59200000e+03,  1.59100000e+03, ...,
        -6.28338067e-02,  6.28930818e-02, -6.28338067e-04],
       ...,
       [ 3.98370000e+04,  3.98500000e+04,  3.97820000e+04, ...,
        -1.35647739e-01,  2.40201258e+03, -1.38157999e-03],
       [ 3.97810000e+04,  3.98170000e+04,  3.97460000e+04, ...,
        -4.77717025e-02,  2.40081761e+03, -4.52579712e-04],
       [ 3.97660000e+04,  3.98250000e+04,  3.97660000e+04, ...,
         1.55802414e-01,  2.40471698e+03,  1.48257996e-03]])

In [21]:
#train_start = 0
#train_end = int(len(dataset) - 1440)
#test_start = int(train_end + 1)
#test_end = int(len(dataset))
#data_train = data[np.arange(train_start, train_end), :]
#data_test = data[np.arange(test_start, test_end), :]

In [12]:
scaler = MinMaxScaler(feature_range = (-1, 1))
training_set_scaled = scaler.fit_transform(training_set)
testing_set_scaled = scaler.transform(testing_set)
validation_set_scaled = scaler.transform(validation_set)

#data_train_scaled = scaler.transform(data_train)
#data_test_scaled = scaler.transform(data_test)

In [17]:
data_tot_norm = np.concatenate((training_set_scaled, validation_set_scaled, testing_set_scaled), axis = 0)
data_tot_norm.shape

(1401894, 89)

In [ ]:
seq_len = 30

def load_data(Crypto, seq_len):
  data_raw = Crypto
  data = []
  for index in range(len(data_raw) - seq_len):
    data.append(data_raw[index: index + seq_len])

  data = np.array(data)
  valid_set_size = 1440
  test_set_size = 1440
  train_set_size = data[0] - (test_set_size + valid_set_size)
  X_train = data[:train_set_size, 5:, :]
  y_train = data[:train_set_size, 3, :]
  X_valid = data[train_set_size:train_set_size+valid_set_size, 5:, :]
  y_valid = data[train_set_size:train_set_size+valid_set_size, 3, :]
  X_test = data[train_set_size+valid_set_size:, 5:, :]
  y_test = data[train_set_size+valid_set_size:, 3, :]

  return [X_train, y_train, X_valid, y_valid, X_test, y_test]

X_train, y_train, X_valid, y_valid, X_test, y_test = load_data(data_tot_norm_df, seq_len)

print('X_train.shape =', X_train.shape)
print('y_train.shape =', y_train.shape)
print('X_valid.shape =', X_valid.shape)
print('y_valid.shape =', y_valid.shape)
print('X_test.shape =', X_test.shape)
print('y_test.shape =', y_test.shape)

In [ ]:
n_steps = seq_len-1
n_inputs = 84
n_neurons = 200
n_outputs = 1
n_layers = 2
learning_rate = 0.001
batch_szie = 50
n_epochs = 100
train_set_size = X_train.shape[0]
test_set_size = X_test.shape[0]
tf.compat.v1.reset_default_graph()
X = tf.compat.v1.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.compat.v1.placeholder(tf.float32, [None, n_outputs])

In [ ]:
index_in_epoch = 0 
perm_array = np.arange(X_train.shape[0])
np.random.shuffle(perm_array)

def get_next_batch(batch_size):
  global index_in_epoch, X_train, perm_array
  start = index_in_epoch
  index_in_epoch += batch_size
  if index_in_epoch > X_train.shape[0]:
    np.random.shuffle(perm_array)
    start = 0
    index_in_epoch = batch_size
  end = index_in_epoch
  return X_train[perm_array[start:end]], y_train[perm_array[start:end]]

In [ ]:
layers = [tf.compat.v1.nn.rnn_cell.GRUCell(num_units = n_neurons, activation = tf.nn.leaky_relu) for layer in range(n_layers)]

multi_layer_cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell(layers)
rnn_outputs, states = tf.compat.v1.nn.dynamic_rnn(multi_layer_cell, X, dtype = tf.float32)
stacked_rnn_ouputs = tf.reshape(rnn_outputs, [-1, n_neurons])
stacked_ouputs = tf.compat.v1.layers.dense(stacked_rnn_ouputs, n_ouputs)
outputs = tf.reshape(stacked_ouputs, [-1, n_steps, n_ouputs])
outputs = outputs[:, n_steps-1, :] #Only last output of the sequence

In [ ]:
loss = tf.compat.v1.reduce_mean(tf.math.square(outputs - y))

In [ ]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)

In [ ]:
saver = tf.compat.v1.train.Saver()
with tf.compat.v1.Session() as sess:
  sess.run(tf.compat.v1.global_variables_initializer())
  for iteration in range(int(n_epochs*train_set_size/batch_size)):
    X_batch, y_batch = get_next_batch(batch_size)
    sess.run(training_op, feed_dict = {X: X_batch, y: y_batch})
    if iteration % int(5*train_set_size/batch_size) == 0:
      mse_train = loss.eval(feed_dict = {X: X_train, y: y_train})
      mse_valid = loss.eval(feed_dict = {X: X_valid, y: y_valid})
      print('%.2f epochs: MSE train/valid = %.6f/%.6f'%(iteration*batch_size/train_set_size, mse_train, mse_valid))
  

  #Predictions
  y_test_pred = sess.run(outputs, feed_dict = {X: X_test})
  saved = saver.save(sess, 'Close_model.ckpt')
  print("Model saved in file: Close_model.ckpt")

In [ ]:
print(y_test_pred)

In [ ]:
seq_len = 30

def load_data(Crypto, seq_len):
  data_raw = Crypto
  data = []
  for index in range(len(data_raw) - seq_len):
    data.append(data_raw[index: index + seq_len])

  data = np.array(data)
  valid_set_size = 1440
  test_set_size = 1440
  train_set_size = data[0] - (test_set_size + valid_set_size)
  X_train = data[:train_set_size, 5:, :]
  y_train = data[:train_set_size, 0, :]
  X_valid = data[train_set_size:train_set_size+valid_set_size, 5:, :]
  y_valid = data[train_set_size:train_set_size+valid_set_size, 0, :]
  X_test = data[train_set_size+valid_set_size:, 5:, :]
  y_test = data[train_set_size+valid_set_size:, 0, :]

  return [X_train, y_train, X_valid, y_valid, X_test, y_test]

X_train, y_train, X_valid, y_valid, X_test, y_test = load_data(data_tot_norm, seq_len)

print('X_train.shape =', X_train.shape)
print('y_train.shape =', y_train.shape)
print('X_valid.shape =', X_valid.shape)
print('y_valid.shape =', y_valid.shape)
print('X_test.shape =', X_test.shape)
print('y_test.shape =', y_test.shape)

In [ ]:
n_steps = seq_len-1
n_inputs = 84
n_neurons = 200
n_outputs = 1
n_layers = 2
learning_rate = 0.001
batch_szie = 50
n_epochs = 100
train_set_size = X_train.shape[0]
test_set_size = X_test.shape[0]
tf.compat.v1.reset_default_graph()
X = tf.compat.v1.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.compat.v1.placeholder(tf.float32, [None, n_outputs])

In [ ]:
index_in_epoch = 0 
perm_array = np.arange(X_train.shape[0])
np.random.shuffle(perm_array)

def get_next_batch(batch_size):
  global index_in_epoch, X_train, perm_array
  start = index_in_epoch
  index_in_epoch += batch_size
  if index_in_epoch > X_train.shape[0]:
    np.random.shuffle(perm_array)
    start = 0
    index_in_epoch = batch_size
  end = index_in_epoch
  return X_train[perm_array[start:end]], y_train[perm_array[start:end]]

In [ ]:
layers = [tf.compat.v1.nn.rnn_cell.GRUCell(num_units = n_neurons, activation = tf.nn.leaky_relu) for layer in range(n_layers)]

multi_layer_cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell(layers)
rnn_outputs, states = tf.compat.v1.nn.dynamic_rnn(multi_layer_cell, X, dtype = tf.float32)
stacked_rnn_ouputs = tf.reshape(rnn_outputs, [-1, n_neurons])
stacked_ouputs = tf.compat.v1.layers.dense(stacked_rnn_ouputs, n_ouputs)
outputs = tf.reshape(stacked_ouputs, [-1, n_steps, n_ouputs])
outputs = outputs[:, n_steps-1, :] #Only last output of the sequence

In [ ]:
loss = tf.compat.v1.reduce_mean(tf.math.square(outputs - y))

In [ ]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)

In [ ]:
saver = tf.compat.v1.train.Saver()
with tf.compat.v1.Session() as sess:
  sess.run(tf.compat.v1.global_variables_initializer())
  for iteration in range(int(n_epochs*train_set_size/batch_size)):
    X_batch, y_batch = get_next_batch(batch_size)
    sess.run(training_op, feed_dict = {X: X_batch, y: y_batch})
    if iteration % int(5*train_set_size/batch_size) == 0:
      mse_train = loss.eval(feed_dict = {X: X_train, y: y_train})
      mse_valid = loss.eval(feed_dict = {X: X_valid, y: y_valid})
      print('%.2f epochs: MSE train/valid = %.6f/%.6f'%(iteration*batch_size/train_set_size, mse_train, mse_valid))
  

  #Predictions
  y_test_pred = sess.run(outputs, feed_dict = {X: X_test})
  saved = saver.save(sess, 'Open_model.ckpt')
  print("Model saved in file: Open_model.ckpt")

In [ ]:
print(y_test_pred)